In [2]:
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [14]:
df = pd.read_csv('compras.csv')

In [15]:
df.head()

,cliente,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,JÚLIO DEMARQUE,limão,peito de peru,manga,cerveja,picanha,carvão,sal,linguiça,mel,geleia,cereal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MAYCON SOARES,salsicha,cerveja,picanha,catchup,geleia,cebola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WEMERSON NETO,queijo,frutas,chá,alface,granola,feijão,farinha de trigo,suco de laranja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JOÃO DEMARQUE,pimenta,sardinha enlatada,barra de cereal,molho de tomate,gelatina,vinho branco,pão,tempero pronto,vinho tinto,presunto,arroz,feijão,NaN,NaN,NaN,NaN,NaN,NaN
4,MAYCON SANTOS,detergente,filé de frango,arroz,feijão,limão,papel higiênico,presunto,farinha de trigo,salada,alface,cenoura,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df = df.drop(columns=['cliente'])
replaces = {'vinho tinto': 'vinho', 'vinho branco': 'vinho', 'alface': 'verdura', 'cerveja artesanal': 'cerveja',}
df.replace(to_replace=replaces, inplace=True)

In [17]:
df.stack().unique()

array(['limão', 'peito de peru', 'manga', 'cerveja', 'picanha', 'carvão',
       'sal', 'linguiça', 'mel', 'geleia', 'cereal', 'salsicha',
       'catchup', 'cebola', 'queijo', 'frutas', 'chá', 'verdura',
       'granola', 'feijão', 'farinha de trigo', 'suco de laranja',
       'pimenta', 'sardinha enlatada', 'barra de cereal',
       'molho de tomate', 'gelatina', 'vinho', 'pão', 'tempero pronto',
       'presunto', 'arroz', 'detergente', 'filé de frango',
       'papel higiênico', 'salada', 'cenoura', 'maionese', 'molho branco',
       'whisky', 'pudim', 'suco', 'azeitona', 'frango', 'mostarda',
       'carne moída', 'nhoque', 'água de coco', 'laranja',
       'massa de pizza', 'biscoito', 'morango', 'bisnaguinha', 'kiwi',
       'achocolatado', 'sabão', 'torrada', 'banana', 'palmito',
       'desinfetante', 'sopa pronta', 'ervilha', 'hambúrguer',
       'batata frita', 'refrigerante', 'queijo ralado', 'uva',
       'óleo de soja', 'vodka', 'batata', 'bolo', 'açúcar', 'café',
       

In [18]:
mais_comprados = df.stack().value_counts()
mais_comprados.head(30)

cerveja         1340
queijo          1241
vinho           1178
presunto        1129
pão             1056
feijão          1040
picanha          869
arroz            856
salada           837
manteiga         803
leite            722
bolo             716
salsicha         682
açúcar           631
sorvete          621
verdura          618
carvão           612
batata frita     607
tomate           577
refrigerante     573
frango           572
cenoura          554
azeite           522
catchup          521
vodka            510
geleia           493
vinagre          479
mostarda         472
água tônica      433
óleo de soja     416
Name: count, dtype: int64

In [20]:
transactions = []
for i in range(0,df.shape[0]):
    transactions.append([str(df.values[i,j])for j in range(0,df.shape[1])])

In [21]:
regras = apriori(transactions=transactions, min_support=0.003, min_confidence=0.2, min_lift=3, min_length=2, max_length=2)

In [22]:
resultados = list(regras)
resultados

[RelationRecord(items=frozenset({'feijão', 'arroz'}), support=0.0544, ordered_statistics=[OrderedStatistic(items_base=frozenset({'arroz'}), items_add=frozenset({'feijão'}), confidence=0.6384976525821596, lift=6.223174001775435), OrderedStatistic(items_base=frozenset({'feijão'}), items_add=frozenset({'arroz'}), confidence=0.530214424951267, lift=6.223174001775435)]),
 RelationRecord(items=frozenset({'filé de frango', 'arroz'}), support=0.0249, ordered_statistics=[OrderedStatistic(items_base=frozenset({'arroz'}), items_add=frozenset({'filé de frango'}), confidence=0.29225352112676056, lift=7.731574632983084), OrderedStatistic(items_base=frozenset({'filé de frango'}), items_add=frozenset({'arroz'}), confidence=0.6587301587301587, lift=7.731574632983084)]),
 RelationRecord(items=frozenset({'frango', 'arroz'}), support=0.0425, ordered_statistics=[OrderedStatistic(items_base=frozenset({'arroz'}), items_add=frozenset({'frango'}), confidence=0.49882629107981225, lift=8.813185354767), OrderedSt

In [23]:
df_compras_lista = df.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(df_compras_lista).transform(df_compras_lista)

# Converte o array resultante em um DataFrame pandas
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# Agora você pode aplicar o fpgrowth no DataFrame one-hot
frequent_itemsets = fpgrowth(df_onehot, min_support=0.003,use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)

rules.size

17000

In [24]:
rules = rules.sort_values(by='lift', ascending=False)
rules.head(30)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
886,"(catchup, hambúrguer)","(mostarda, batata frita, cerveja)",0.0145,0.0057,0.0042,0.289655,50.816697,0.004117,1.399743,0.994745
871,"(mostarda, batata frita, cerveja)","(catchup, hambúrguer)",0.0057,0.0145,0.0042,0.736842,50.816697,0.004117,3.744900,0.985941
876,"(catchup, batata frita, cerveja)","(hambúrguer, mostarda)",0.0076,0.0110,0.0042,0.552632,50.239234,0.004116,2.210706,0.987601
881,"(hambúrguer, mostarda)","(catchup, batata frita, cerveja)",0.0110,0.0076,0.0042,0.381818,50.239234,0.004116,1.605353,0.990996
1297,"(azeite, salada)","(tomate, cenoura, vinagre)",0.0151,0.0054,0.0040,0.264901,49.055678,0.003918,1.353014,0.994634
1292,"(tomate, cenoura, vinagre)","(azeite, salada)",0.0054,0.0151,0.0040,0.740741,49.055678,0.003918,3.798900,0.984934
1499,"(verdura, vinagre, salada)","(tomate, azeite)",0.0078,0.0111,0.0042,0.538462,48.510049,0.004113,2.142617,0.987085
1514,"(tomate, azeite)","(verdura, vinagre, salada)",0.0111,0.0078,0.0042,0.378378,48.510049,0.004113,1.596148,0.990379
1287,"(vinagre, cenoura, salada)","(tomate, azeite)",0.0076,0.0111,0.0040,0.526316,47.415837,0.003916,2.087678,0.986407
1302,"(tomate, azeite)","(vinagre, cenoura, salada)",0.0111,0.0076,0.0040,0.360360,47.415837,0.003916,1.551499,0.989898
